## Dnyanai Surkutwar <br><ol><li>Part I: Creating Dataset</li><li>Part II: Data Preprocessing</li><li>Part III: Extracting data from PDFs or web scrape the site's content(?)</li>

## Importing libraries

In [1]:
import pandas as pd 
import tweepy as tw 

# Table of Content: 

1.[Extracting Tweets from Twitter user - @SalesforceNews](#tweepy) <br>
   * [Setting up twitter app authentication](#tweepy)
   * [Creating API object](#auth)
   * [Collecting tweets](#collect)
   * [Extracting relevant information from the tweets](#extract)
   * [Creating Dataframe of extracted tweets](#df)
   * [Creating labels : 1)Case Studies 2)Press Release](#labels) <br>

2.[Data Preprocessing](#preproc) <br>
   * [Cleaning the data](#clean)
   * [Vectorize, tokenize the data](#token)


<a id = 'tweepy'></a>
# Extracting tweets using tweepy

In [2]:
#Authentication for twitter:
consumer_key= 'muauHMRfmPUXau1XDaUqdFMJQ'
consumer_secret= 'j3afAVjrB7H11n84RMxXqEj9ygYNcgDzwtZeMcmTXuri1j1Wth'
access_token= '762032959-5Z5J7Dlznx8kJ485B9dRU2NWSR2ukYiwilQ0W9KO'
token_secret= 'dx70UrIpiLZLW9kurUky3CeppoBH6zaqhdICtY0ymNIyH'

<a id = 'auth'></a>
## Creating API object:

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, token_secret)

api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
screen_name = 'SalesforceNews'

<a id='collect'></a>
## Collecting tweets

In [5]:
output = [status._json for status in tw.Cursor(api.search, q='SalesforceNews -filter:retweets',\
                                                count=100, tweet_mode='extended', include_entities=True, lang='en').items()]

In [6]:
#import pickle
#filename = 'tweets_100_Dec_3.pkl'
#out = open(filename,'wb')
#pickle.dump(output,out)
#out.close()

In [7]:
output[2]['entities']['urls']

[]

<a id='extract'></a>
## Extracting the relevant information from the tweets

In [8]:
full_text = []   ## saving the tweet text in a list
post_url = []    ## saving the urls in a list 
tw_id = []       ## saving the tweet id in a list

for each in output:
    tw_id.append(each['id_str'])
    full_text.append(each['full_text'])
    post_url.append(each['entities']['urls'])

In [9]:
#post_url[0][0]['display_url'] ## testing different keys in the urls 

In [10]:
#post_url[0][0]['expanded_url']  ## We need the expanded url
urls = []   ## creating new list to save expanded urls only
disp_urls = [] ## we might get more information from display urls too, so creating a new list for that

for i in range(len(post_url)):
    #print(post_url[i])
    if post_url[i] != []:
        urls.append(post_url[i][0]['expanded_url'])
        disp_urls.append(post_url[i][0]['display_url'])
    else:
        urls.append('')
        disp_urls.append('')
disp_urls[:5]

['',
 '',
 '',
 'salesforce.com/news/stories/t…',
 'salesforce.com/news/stories/d…']

In [11]:
full_text[2]

'@SalesforceNews @salesforce Why would any employee want to be spied on. \nAs an employee I demand to know what the boss is doing .'

<a id='#df'></a>
## Creating a Dataset dataframe from the collected tweets: 

In [12]:
data = {'tweet_id': tw_id, 'full_text':full_text, 'url':urls, 'disp_url':disp_urls}

df_salesforce = pd.DataFrame(data=data)

In [13]:
df_salesforce.head()

,tweet_id,full_text,url,disp_url
0,1335003258910334978,@SalesforceNews @salesforce Salesforce is the ...,,
1,1334917749517275136,@OzzNationalist @SalesforceNews @salesforce We...,,
2,1334917487050240003,@SalesforceNews @salesforce Why would any empl...,,
3,1334817030659338244,The Biggest News from Dreamforce to You https:...,https://www.salesforce.com/news/stories/the-bi...,salesforce.com/news/stories/t…
4,1334681871930814464,Dreamforce to You 2020: Reimagine Everything h...,https://www.salesforce.com/news/stories/dreamf...,salesforce.com/news/stories/d…


<a id='labels'></a>
## Creating labels based on the url redirect -> stories: Case Studies, press-releases : Press Releases. This is specific to Salesforce. 

In [14]:
df_salesforce['labels'] = ['case studies' if 'stories' in each else '' for each in df_salesforce.url]
df_salesforce.loc[df_salesforce.url.str.contains('press-releases')==True,'labels'] = 'press release'
df_salesforce.loc[df_salesforce.url.str.contains('moments')==True,'labels'] = 'press release' # few urls are moments which are referring to press releases so we manually updated them
df_salesforce.loc[df_salesforce.url.str.contains('status')==True,'labels'] = 'press release' # few urls are status which are referring to press releases so we manually updated them

## WIP:

## Some of the urls are not straightforward so we will use requests package to ping them and see their url address, to check if they can be categorized into 'case studies' or 'press release': 

In [15]:
df_salesforce.url.loc[4]

'https://www.salesforce.com/news/stories/dreamforce-to-you-2020/'

In [36]:
s = 'https://www.salesforce.com/news/stories/dreamforce-to-you-2020/'

df_salesforce.labels[df_salesforce.url==s].index[0]

4

In [26]:
pd.set_option('max.rows',None)
df_salesforce

,tweet_id,full_text,url,disp_url,labels
0,1335003258910334978,@SalesforceNews @salesforce Salesforce is the ...,,,
1,1334917749517275136,@OzzNationalist @SalesforceNews @salesforce We...,,,
2,1334917487050240003,@SalesforceNews @salesforce Why would any empl...,,,
3,1334817030659338244,The Biggest News from Dreamforce to You https:...,https://www.salesforce.com/news/stories/the-bi...,salesforce.com/news/stories/t…,case studies
4,1334681871930814464,Dreamforce to You 2020: Reimagine Everything h...,https://www.salesforce.com/news/stories/dreamf...,salesforce.com/news/stories/d…,case studies
5,1334673337591754755,Introducing Salesforce Hyperforce: the world’s...,https://www.salesforce.com/news/press-releases...,salesforce.com/news/press-rel…,press release
6,1334636358414049282,Salesforce Introduces Service Cloud Workforce ...,https://www.salesforce.com/news/press-releases...,salesforce.com/news/press-rel…,press release
7,1334608137018630146,The Biggest News from Dreamforce to You https:...,https://www.salesforce.com/news/stories/the-bi...,salesforce.com/news/stories/t…,case studies
8,1334598276381753344,"""The future of automation is human"" \n\nAutoma...",,,
9,1334587871823601664,⚡️ “Dreamforce to You: Keynote Highlights”\n\n...,https://twitter.com/i/moments/1334581347143331843,twitter.com/i/moments/1334…,press release


In [43]:
import requests
import time

cs_url = [] 
cs_id = []
pr_url = []
pr_id = []

for site in range(len(df_salesforce[df_salesforce.labels==''].url)):

    if df_salesforce.url[site]!='':
        res=requests.get(df_salesforce.url[site],timeout=10)
        
        if 'stories' in res.url:
            print(res.url)
            print(df_salesforce.url[site])
            #cs_id.append(df_salesforce.labels[df_salesforce.url==res.url].index[0])
            print(count_cs,': cs-done :',res.status_code)
            
            
        elif 'press-releases' in res.url:
            print(res.url)
            #pr_id.append(df_salesforce.labels[df_salesforce.url==str(res.url)].index)
            print(count_pr,': pr-done',res.status_code)
            
        #else:
        #    pass


https://www.salesforce.com/news/stories/the-biggest-news-of-dreamforce-to-you/?bc=HL
Series([], Name: url, dtype: object)
1 : cs-done : 200
https://www.salesforce.com/news/stories/dreamforce-to-you-2020/?bc=HL
Series([], Name: url, dtype: object)
1 : cs-done : 200
https://www.salesforce.com/news/press-releases/2020/12/02/introducing-salesforce-hyperforce/?bc=HL
39 : pr-done 200
https://www.salesforce.com/news/press-releases/2020/12/02/service-cloud-workforce-engagement/?bc=HL
39 : pr-done 200
https://www.salesforce.com/news/stories/the-biggest-news-of-dreamforce-to-you/?bc=HL
Series([], Name: url, dtype: object)
1 : cs-done : 200
https://www.salesforce.com/news/stories/salesforce-recognizes-2020-international-day-of-people-with-disabilities/?utm_source=twitter&utm_medium=organic_social&utm_campaign=amer_cbaw&utm_content=AMER%2CEnglish%2CNewsroom%2CPartner+Mentioned%2CPartner+Tagged&bc=HL
Series([], Name: url, dtype: object)
1 : cs-done : 200
https://www.salesforce.com/news/stories/fort

In [31]:
'stories' in 'https://www.salesforce.com/news/stories/the-bi'

True

In [ ]:
pd.set_option('max.rows',None)
df_salesforce

In [ ]:
len(df_salesforce[df_salesforce.labels==''])

In [ ]:
len(df_salesforce[df_salesforce.labels==''])

[BS for scraping, using selectorGadget]